In [3]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from scipy import sparse
import scipy


import sys
sys.path.append('..')

In [4]:
L = 6
# Sz = np.zeros((3,3),dtype = np.complex128)
# Sx = np.zeros((3,3),dtype = np.complex128)
# Sy = np.zeros((3,3),dtype = np.complex128)

# Sz[0,0] = 1
# Sz[2,2] = -1
# Sx[0,1] = 1/np.sqrt(2)
# Sx[1,0] = 1/np.sqrt(2)
# Sx[1,2] = 1/np.sqrt(2)
# Sx[2,1] = 1/np.sqrt(2)
# Sy[0,1] = -1j/np.sqrt(2)
# Sy[1,0] = 1j/np.sqrt(2)
# Sy[1,2] = -1j/np.sqrt(2)
# Sy[2,1] = 1j/np.sqrt(2)

Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2

# SzSz = sparse.kron(Sz,Sz,format='csr')
# SxSx = sparse.kron(Sx,Sx,format='csr')
# SySy = sparse.kron(Sy,Sy,format='csr')

# h = SzSz + SxSx + SySy

SzSz = np.kron(Sz,Sz).astype(np.float64)
SxSx = np.kron(Sx,Sx).astype(np.float64)
SySy = np.kron(Sy,Sy).astype(np.float64)

lh = SzSz + SxSx + SySy

/home/keisuke/miniconda3/envs/quspin/lib/python3.7/site-packages/ipykernel_launcher.py:35: ComplexWarning: Casting complex values to real discards the imaginary part


### exact diagonalization with original matrix

In [45]:
import functions as f
from importlib import reload
from scipy import sparse
import scipy.sparse.linalg
import scipy
reload(f)

H = sparse.csr_matrix((2**L, 2**L), dtype=np.float64)
for i in range(L):
    
    H += f.l2nl(lh, L, [i,(i+1)%L], sps = 2)
    H += f.l2nl(lh/2, L, [i,(i+2)%L], sps = 2)   

E, V = np.linalg.eigh(H.toarray())
print(E)

[-2.25000000e+00 -2.25000000e+00 -1.75000000e+00 -1.75000000e+00
 -1.75000000e+00 -1.56872930e+00 -1.56872930e+00 -1.56872930e+00
 -1.56872930e+00 -1.56872930e+00 -1.56872930e+00 -1.25000000e+00
 -1.25000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -2.50000000e-01
 -2.50000000e-01 -2.50000000e-01 -2.50000000e-01 -2.50000000e-01
 -2.50000000e-01 -3.51510989e-16 -2.90853943e-16  5.92721140e-17
  1.09018292e-16  1.61187079e-16  1.69567322e-16  2.27804931e-16
  4.27895823e-16  5.70719320e-16  7.03265248e-16  2.50000000e-01
  2.50000000e-01  2.50000000e-01  2.50000000e-01  2.50000000e-01
  2.50000000e-01  3.18729304e-01  3.18729304e-01  3.18729304e-01
  3.18729304e-01  3.18729304e-01  3.18729304e-01  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  2.25000000e+00  2.25000000e+00  2.25000000e+00
  2.25000000e+00  2.25000

/home/keisuke/miniconda3/envs/quspin/lib/python3.7/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


### rewritten with projection operators

In [7]:
LH = sparse.csr_matrix((2**3,2**3), dtype = np.float64)
i = 0
LH += f.l2nl(lh/2, 3, [i,(i+1)%L], sps = 2)
LH += f.l2nl(lh/2, 3, [(i+1)%L,(i+2)%L], sps = 2)
LH += f.l2nl(lh/2, 3, [i,(i+2)%L], sps = 2) 
LH = LH

In [8]:
LH

<8x8 sparse matrix of type '<class 'numpy.complex128'>'
	with 20 stored elements in Compressed Sparse Row format>

In [38]:
LH_ = sparse.csr_matrix((2**6,2**6), dtype = np.float64)
i = 0
LH_ += f.l2nl(lh/2, 6, [0, 1], sps = 2)
LH_ += f.l2nl(lh/2, 6, [1, 2], sps = 2)
LH_ += f.l2nl(lh, 6, [2, 3], sps = 2)
LH_ += f.l2nl(lh/2, 6, [3, 4], sps = 2)
LH_ += f.l2nl(lh/2, 6, [4, 5], sps = 2)
LH_ += f.l2nl(lh/4, 6, [0, 2], sps = 2)
LH_ += f.l2nl(lh/2, 6, [1, 3], sps = 2)
LH_ += f.l2nl(lh/2, 6, [2, 4], sps = 2)
LH_ += f.l2nl(lh/4, 6, [3, 5], sps = 2)


# LH_ += f.l2nl(lh/2, 6, [(0 + 3)%6, (1 + 3 ) % 6], sps = 2)
# LH_ += f.l2nl(lh/2, 6, [(1 + 3)%6, (2 + 3 ) % 6], sps = 2)
# LH_ += f.l2nl(lh/2, 6, [(2 + 3)%6, (3 + 3 ) % 6], sps = 2)
# LH_ += f.l2nl(lh, 6, [(3 + 3)%6, (4 + 3 ) % 6], sps = 2)
# LH_ += f.l2nl(lh/2, 6, [(4 + 3)%6, (5 + 3 ) % 6], sps = 2)
# LH_ += f.l2nl(lh/4, 6, [(0 + 3)%6, (2 + 3 ) % 6], sps = 2)
# LH_ += f.l2nl(lh/2, 6, [(1 + 3)%6, (3 + 3 ) % 6], sps = 2)
# LH_ += f.l2nl(lh/2, 6, [(2 + 3)%6, (4 + 3 ) % 6], sps = 2)
# LH_ += f.l2nl(lh/4, 6, [(3 + 3)%6, (5 + 3 ) % 6], sps = 2)

In [39]:
H = sparse.csr_matrix((2**6,2**6), dtype = np.float64)
H += f.l2nl(LH_, 2, [0, 1], sps = 2**3)
H += f.l2nl(LH_, 2, [1, 0], sps = 2**3)

In [40]:
E2, V2 = np.linalg.eigh(H.toarray())
print((E2).round(6))

[-2.25     -2.25     -1.75     -1.75     -1.75     -1.568729 -1.568729
 -1.568729 -1.568729 -1.568729 -1.568729 -1.25     -1.25     -1.
 -1.       -1.       -1.       -1.       -1.       -0.25     -0.25
 -0.25     -0.25     -0.25     -0.25     -0.       -0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.25      0.25      0.25      0.25      0.25      0.25      0.318729
  0.318729  0.318729  0.318729  0.318729  0.318729  1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        2.25      2.25      2.25      2.25      2.25      2.25
  2.25    ]


In [71]:
X = -LH_.toarray()
X -= np.eye(X.shape[0]) * np.min(np.diag(X))

In [50]:
def callbackF(x, f, context):
    print("target value : {:.5f} in the context {}".format(f,context))

In [77]:
np.linalg.eigvalsh(X)

array([0.000e+00, 0.000e+00, 1.252e-16, 1.378e-16, 2.027e-16, 2.629e-16,
       4.061e-16, 2.409e-01, 2.409e-01, 2.409e-01, 2.409e-01, 2.409e-01,
       5.410e-01, 5.410e-01, 5.410e-01, 5.443e-01, 5.443e-01, 5.443e-01,
       5.443e-01, 5.443e-01, 8.710e-01, 8.710e-01, 8.710e-01, 8.710e-01,
       8.710e-01, 8.725e-01, 8.725e-01, 8.725e-01, 9.851e-01, 1.083e+00,
       1.083e+00, 1.083e+00, 1.206e+00, 1.206e+00, 1.206e+00, 1.206e+00,
       1.206e+00, 1.481e+00, 1.481e+00, 1.481e+00, 1.518e+00, 1.518e+00,
       1.518e+00, 1.638e+00, 1.638e+00, 1.638e+00, 1.638e+00, 1.638e+00,
       1.686e+00, 1.710e+00, 1.710e+00, 1.710e+00, 1.750e+00, 1.925e+00,
       1.984e+00, 1.984e+00, 1.984e+00, 2.147e+00, 2.147e+00, 2.147e+00,
       2.164e+00, 2.164e+00, 2.164e+00, 2.654e+00])

In [95]:
from utils import optm
reload(optm)
# X = np.stack([X1_prime, X2_prime])
func = optm.unitary_optm(X)
import scipy.optimize as optimize
bounds = [[0, 2*np.pi] for _ in range(28)]
ret = optimize.dual_annealing(func, bounds = bounds, restart_temp_ratio = 1e-5, visit = 2.68, initial_temp = 3*10**4, maxiter = 50000, callback = callbackF)
print(ret)

[28]
target value : 5.03624 in the context 0
target value : 5.00251 in the context 0
target value : 4.99281 in the context 0
target value : 4.65189 in the context 0
target value : 4.56447 in the context 0
target value : 4.45966 in the context 0
target value : 4.37529 in the context 0
target value : 4.34073 in the context 0
target value : 4.20885 in the context 0
target value : 4.20845 in the context 0
target value : 4.13394 in the context 0
target value : 2.93446 in the context 1
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the context 2
target value : 2.93446 in the conte

target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2


target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2


target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 1
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2
target value : 2.82490 in the context 2


In [94]:
reload(optm)
import torch.optim
model, gl = optm.optim_matrix_symm([torch.tensor(X)], 100000, optm_method = torch.optim.SGD, seed = 10, lr = 0.01)

target loss : 2.654

---------- iteration start ----------
iteration :    0   loss : 4.991
iteration : 1000   loss : 2.861
iteration : 2000   loss : 2.861
iteration : 3000   loss : 2.861
iteration : 4000   loss : 2.861
iteration : 5000   loss : 2.861
iteration : 6000   loss : 2.861
iteration : 7000   loss : 2.861
iteration : 8000   loss : 2.861
iteration : 9000   loss : 2.861
iteration : 10000   loss : 2.861
iteration : 11000   loss : 2.861
iteration : 12000   loss : 2.861
iteration : 13000   loss : 2.861
iteration : 14000   loss : 2.861
iteration : 15000   loss : 2.861
iteration : 16000   loss : 2.861
iteration : 17000   loss : 2.861
iteration : 18000   loss : 2.861
iteration : 19000   loss : 2.861
iteration : 20000   loss : 2.861
iteration : 21000   loss : 2.861
iteration : 22000   loss : 2.862
iteration : 23000   loss : 2.861
iteration : 24000   loss : 2.861
iteration : 25000   loss : 2.861
iteration : 26000   loss : 2.861
iteration : 27000   loss : 2.861
iteration : 28000   loss : 

In [61]:
np.linalg.eigvalsh(X)

array([-1.529, -1.039, -1.039, -1.039, -1.022, -1.022, -1.022, -0.859,
       -0.859, -0.859, -0.8  , -0.625, -0.585, -0.585, -0.585, -0.561,
       -0.513, -0.513, -0.513, -0.513, -0.513, -0.393, -0.393, -0.393,
       -0.356, -0.356, -0.356, -0.081, -0.081, -0.081, -0.081, -0.081,
        0.042,  0.042,  0.042,  0.14 ,  0.253,  0.253,  0.253,  0.254,
        0.254,  0.254,  0.254,  0.254,  0.581,  0.581,  0.581,  0.581,
        0.581,  0.584,  0.584,  0.584,  0.884,  0.884,  0.884,  0.884,
        0.884,  1.125,  1.125,  1.125,  1.125,  1.125,  1.125,  1.125])

In [160]:
H = sparse.csr_matrix((2**L, 2**L), dtype=np.float64)
for i in range(L):
    
    H += f.l2nl(LH, L, [i,(i+1)%L,(i+2)%L], sps = 2) 
    
E, _ = scipy.sparse.linalg.eigsh(H, k = 10)
print(E)

[-2.25 -2.25  2.25 -1.75  2.25 -1.75  2.25  2.25  2.25  2.25]


### apply local unitary transformation

In [161]:
LH_prime = (LH + sparse.identity(2**3) * 3/8).toarray()
E, V = np.linalg.eigh(LH_prime)
V = V.real
print((E).round(6))

[-0.   -0.    0.    0.    0.75  0.75  0.75  0.75]


In [162]:
LH2 = V.T @ LH_prime @ V

In [167]:
U = np.kron(V, V)
C1 = sparse.csr_matrix((2**6, 2**6), dtype=np.float64)
C1 += f.l2nl(LH_prime, L, [1,2,3], sps = 2)
C1 += f.l2nl(LH_prime, L, [2,3,4], sps = 2)
C1 = C1.real.toarray()
C2 = U.T @ C1 @ U

In [168]:
U = np.kron(V, V)
C1_prime = sparse.csr_matrix((2**6, 2**6), dtype=np.float64)
C1_prime += f.l2nl(LH_prime, L, [4,5,0], sps = 2)
C1_prime += f.l2nl(LH_prime, L, [5,0,1], sps = 2)
C1_prime = C1_prime.real.toarray()
C2_prime = U.T @ C1_prime @ U

### symmetrization operator

In [182]:
p = np.zeros((2**3, 2**3))
for state in [[i,j,k] for i in range(2) for j in range(2) for k in range(2)]:
    n = f.state2num(np.array(state), sps = 2)
    m = f.state2num(np.array(state[::-1]), sps = 2)
    p[m,n] += 1/2
    p[n,n] += 1/2  
P = np.kron(p,p)

In [184]:
p

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0. , 0.5, 0. ],
       [0. , 0.5, 0. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])

### apply symmetrization operator to $C1$ and $C1^{\prime}$ so that both become the same operator

In [ ]:
C1@P

In [120]:
index = []
for state in [[i,j] for i in range(4) for j in range(4)]:
    index.append(f.state2num(np.array(state), sps = 8))
index = np.array(index)

In [121]:
C3 = C2[:,index]

In [127]:
U3, E3, V3 = np.linalg.svd(C3)

In [147]:
V4 = np.zeros((4, 2**L))
V4[:, index] = V3[-4:]

In [148]:
C2 @ V4[0]

array([-6.93889390e-18,  1.11022302e-16,  9.71445147e-17,  4.51028104e-17,
       -1.47451495e-17, -5.26922256e-17,  0.00000000e+00,  2.08166817e-17,
       -6.93889390e-18,  5.89805982e-17, -9.36750677e-17,  4.85722573e-17,
       -3.29597460e-17, -3.12250226e-17,  1.73472348e-18,  1.56125113e-17,
       -3.64291930e-17,  7.63278329e-17,  1.24900090e-16,  3.12250226e-17,
        3.46944695e-18, -3.55618313e-17,  2.81491157e-19,  0.00000000e+00,
        3.12250226e-17, -1.45716772e-16, -8.50014503e-17, -3.12250226e-17,
       -7.02563008e-17,  1.38777878e-17, -4.33680869e-18, -3.46944695e-17,
       -2.42861287e-17,  1.12757026e-17,  3.81639165e-17, -2.60208521e-18,
        3.46944695e-18,  4.16333634e-17,  0.00000000e+00, -2.94902991e-17,
        4.16333634e-17, -1.73472348e-17, -3.46944695e-17, -3.46944695e-18,
        3.46944695e-18,  3.46944695e-18,  1.30104261e-17, -3.85185989e-33,
        0.00000000e+00,  6.93889390e-18,  2.16840434e-17,  4.33680869e-18,
        0.00000000e+00,  

In [150]:
V4.shape

(4, 64)